# Assigning Reviewers to Candidates
[J. Nathan Matias](https://natematias.com), January 8, 2021

## Documentation
* Explanation for [how to set up the min cost flow algorithm for committee allocation](http://ozark.hendrix.edu/~yorgey/382/static/flow-network-application.pdf).

* ortools library documentation: [Assignment as a Minimum Cost Flow Problem](https://developers.google.com/optimization/flow/assignment_min_cost_flow)

## Illustration of the Min Cost Flow Diagram Used Here
<img src="flow_graph_illustration.jpg" alt="Drawing" style="width: 500px;"/>

In [254]:
import csv, os, sys, math, datetime
from collections import Counter, defaultdict
from ortools.graph.python import min_cost_flow
import pandas as pd
import random
random.seed(1729711011) #https://www.brooklynintegers.com/int/1729711011/

# Load Data
**anonymized-applicants.csv**: the applicant dataset needs the following columns:
* First Name
* Last Name
* Columns for prioritized reviewers (with valid reviewer ID):
  * 1 top
  * 2
  * 3
  * 4 lowest
  
**anonymized-reviewers.csv** needs the following columns:
* id (associated with the columns for prioritized reviewers)
* Full Review Quota (TRUE) or (FALSE)

In [255]:
# applicant_file = "data/anonymized-applicants.csv"
applicant_file = "data/anonymizedApplicants_v2_test.csv"
applicants = []

with open(applicant_file) as f:
    for row in csv.DictReader(f):
        applicants.append(row)

# randomly shuffle applicants
# in case there are systematic
# patterns in application order
# that would otherwise contribute
# to bias
random.shuffle(applicants)

In [256]:
reviewer_file="data/anonymized-reviewers.csv"
reviewers = []
with open(reviewer_file) as f:
    for row in csv.DictReader(f):
        reviewers.append(row)

        
# randomly shuffle reviewers
# in case there are systematic
# patterns in reviewer order
# that would otherwise contribute
# to bias
random.shuffle(reviewers)

reviewers_full = [x for x in reviewers if x['Full Review Quota']=="TRUE"]
reviewers_occasional = [x for x in reviewers if x['Full Review Quota']!="TRUE"]

# Validate Applicants File
Make sure that all of the recommended reviewers are in the reviewers dataset, and if not, list out applicants where this discrepancy exists.
As the allocator, you can then go into the spreadsheet and make any corrections as needed.

In [257]:
reviewer_netids = [x['id'].lower() for x in reviewers]

validity_record = []

for applicant in applicants:
    applicant_fine = True
    for i in range(1,5):
        if applicant[str(i)] not in reviewer_netids:
            applicant_fine = False
    if(applicant_fine == False):
        print(applicant)
    validity_record.append(applicant_fine)
    
print("\n\n---------------------------\n\n")
print("{0} records have invalid reviewer suggestions".format(len([x for x in validity_record if x!=True])))



---------------------------


0 records have invalid reviewer suggestions


## Basic Statistics

In [258]:
print("{0} total applicants".format(len(applicants)))
print("{0} available faculty for a full round of reviews".format(len(reviewers_full)))
print("{0} reviewers who can take a few".format(len(reviewers_occasional)))
print("Roughly {0} reviews per full reviewer faculty".format(math.floor(len(applicants) * 2 / len(reviewers_full))))

51 total applicants
16 available faculty for a full round of reviews
6 reviewers who can take a few
Roughly 6 reviews per full reviewer faculty


# Set up Graph

### Utility Methods

In [259]:
## Print vertex details for debugging
def print_vertex(i):
    print("{0} -> {1} (capacity {2}, cost {3})".format(
        start_nodes[i], end_nodes[i], 
        capacities[i], costs[i]
    ))

## Calculate the priority of a reviewer for a given candidate
## based on provided information
def reviewer_candidate_priority(applicant, reviewer):
    max_cost_full = 6
    max_cost_partial = 8
    if(reviewer['Full Review Quota']=="TRUE"):
        cost = max_cost_full
    else:
        cost = max_cost_partial
    reviewer_netid = reviewer['id'].lower().strip()
    for i in list(range(1,5)):
        if(reviewer_netid in applicant[str(i)].lower().strip()):
            cost = i
    ##TODO: Calculate topic overlaps to improve precision of matches
    return cost

### Allocation Algorithm Settings

In [260]:
# set up enough capacity to handle all reviews across all full reviewers
# we can use the floor, since there are partial reviewers
full_reviewer_assignment_count = math.floor(len(applicants) * 2 / len(reviewers_full))

# no more than five reviews per partial reviewer
partial_reviewer_assignment_count = 5 

total_tasks = len(applicants) * 2

reviews_per_applicant = 2


### Set up Cost Flow Graph

In [261]:
## All reviewers and candidates need to be given node IDs
## on the same linear scale from 0..n
counter = 1
for reviewer in reviewers:
    reviewer['node_index'] = counter
    counter += 1
    
for applicant in applicants:
    applicant['node_index'] = counter
    counter += 1

## START_NODES AND END_NODES:
## tasks flow from the source (index 0) to the sink (last index)
source_index = source = 0
sink = sink_index = counter

## each of these "nodes" is a node in a vertex. Example:
## start_nodes[0] -> end_nodes[0]
## start_nodes[1] -> end_nodes[1]
## and so forth

start_nodes = []
end_nodes   = []

## CAPACITIES: how many tasks can flow across a vertex
## Each reviewer can take on full_reviewer_assignment_count reviewers
##      from the Source.
## Each applicant can take on only one review from one faculty
## The sink can take two reviews from each applicant
capacities = []

## COSTS: proxy for priority, where higher priority = lower cost
## on a scale from 0 to N
costs = []

## First, add vertices from the source (index 0) to the reviewers
for reviewer in reviewers:
    start_nodes.append(source_index)
    end_nodes.append(reviewer['node_index'])
    if(reviewer['Full Review Quota']=="TRUE"):
        capacities.append(full_reviewer_assignment_count)
    else:
        capacities.append(partial_reviewer_assignment_count)
        
    # no cost to allocate from the source
    costs.append(0)
        
    
## now add vertices from each reviewer to each applicant:
for reviewer in reviewers:
    for applicant in applicants:
        start_nodes.append(reviewer['node_index'])
        end_nodes.append(applicant['node_index'])
        # only one review from each reviewer
        capacities.append(1)
        
        ## cost for a given reviewer applicant pair
        costs.append(reviewer_candidate_priority(applicant, reviewer))
        
## now add vertices from each applicant to the sink
for applicant in applicants:
    start_nodes.append(applicant['node_index'])
    end_nodes.append(sink_index)
    # N applications per candidate
    capacities.append(reviews_per_applicant)
    
    # no cost to reach the sink
    costs.append(0)
    
    
## SET SUPPLIES: This is a vector with a single count
## for the number of supplies available at each node

## set the number of supplies at the source
supplies    = [total_tasks]

## set reviewers and applicant supplies to zero
for reviewer in reviewers:
    supplies.append(0)
for applicant in applicants:
    supplies.append(0)

## set the sink supply to zero
supplies.append(total_tasks*-1)

### Confirm validity of graph

In [262]:
print("{0}: start node, end node, cost, and capacity all have equal length".format(len(start_nodes) == len(end_nodes) == len(costs) == len(capacities)))

True: start node, end node, cost, and capacity all have equal length


In [263]:
## confirm that the source is set up properly
# each link from the source to a reviewer
# should have a cost of 0
for i in range(0,len(reviewers)):
    print_vertex(i)

0 -> 1 (capacity 5, cost 0)
0 -> 2 (capacity 6, cost 0)
0 -> 3 (capacity 6, cost 0)
0 -> 4 (capacity 5, cost 0)
0 -> 5 (capacity 5, cost 0)
0 -> 6 (capacity 6, cost 0)
0 -> 7 (capacity 6, cost 0)
0 -> 8 (capacity 6, cost 0)
0 -> 9 (capacity 6, cost 0)
0 -> 10 (capacity 6, cost 0)
0 -> 11 (capacity 6, cost 0)
0 -> 12 (capacity 6, cost 0)
0 -> 13 (capacity 5, cost 0)
0 -> 14 (capacity 6, cost 0)
0 -> 15 (capacity 6, cost 0)
0 -> 16 (capacity 5, cost 0)
0 -> 17 (capacity 6, cost 0)
0 -> 18 (capacity 5, cost 0)
0 -> 19 (capacity 6, cost 0)
0 -> 20 (capacity 6, cost 0)
0 -> 21 (capacity 6, cost 0)
0 -> 22 (capacity 6, cost 0)


In [264]:
## confirm some of the reviewer to applicant links (reviewer 3)
#  the capacity for each should be 1 and the cost should vary
for i in range(len(reviewers) + len(applicants)*3, len(reviewers) + len(applicants*3) + 20):
    print_vertex(i)

4 -> 23 (capacity 1, cost 8)
4 -> 24 (capacity 1, cost 8)
4 -> 25 (capacity 1, cost 8)
4 -> 26 (capacity 1, cost 8)
4 -> 27 (capacity 1, cost 8)
4 -> 28 (capacity 1, cost 8)
4 -> 29 (capacity 1, cost 8)
4 -> 30 (capacity 1, cost 8)
4 -> 31 (capacity 1, cost 8)
4 -> 32 (capacity 1, cost 8)
4 -> 33 (capacity 1, cost 8)
4 -> 34 (capacity 1, cost 8)
4 -> 35 (capacity 1, cost 8)
4 -> 36 (capacity 1, cost 8)
4 -> 37 (capacity 1, cost 8)
4 -> 38 (capacity 1, cost 8)
4 -> 39 (capacity 1, cost 8)
4 -> 40 (capacity 1, cost 8)
4 -> 41 (capacity 1, cost 8)
4 -> 42 (capacity 1, cost 8)


In [265]:
## confirm that the sink is set up properly
# each applicant should have a capacity of 2 and cost of 0
for i in range(len(start_nodes) - len(applicants), len(start_nodes)-1):
    print_vertex(i)

23 -> 74 (capacity 2, cost 0)
24 -> 74 (capacity 2, cost 0)
25 -> 74 (capacity 2, cost 0)
26 -> 74 (capacity 2, cost 0)
27 -> 74 (capacity 2, cost 0)
28 -> 74 (capacity 2, cost 0)
29 -> 74 (capacity 2, cost 0)
30 -> 74 (capacity 2, cost 0)
31 -> 74 (capacity 2, cost 0)
32 -> 74 (capacity 2, cost 0)
33 -> 74 (capacity 2, cost 0)
34 -> 74 (capacity 2, cost 0)
35 -> 74 (capacity 2, cost 0)
36 -> 74 (capacity 2, cost 0)
37 -> 74 (capacity 2, cost 0)
38 -> 74 (capacity 2, cost 0)
39 -> 74 (capacity 2, cost 0)
40 -> 74 (capacity 2, cost 0)
41 -> 74 (capacity 2, cost 0)
42 -> 74 (capacity 2, cost 0)
43 -> 74 (capacity 2, cost 0)
44 -> 74 (capacity 2, cost 0)
45 -> 74 (capacity 2, cost 0)
46 -> 74 (capacity 2, cost 0)
47 -> 74 (capacity 2, cost 0)
48 -> 74 (capacity 2, cost 0)
49 -> 74 (capacity 2, cost 0)
50 -> 74 (capacity 2, cost 0)
51 -> 74 (capacity 2, cost 0)
52 -> 74 (capacity 2, cost 0)
53 -> 74 (capacity 2, cost 0)
54 -> 74 (capacity 2, cost 0)
55 -> 74 (capacity 2, cost 0)
56 -> 74 (

In [266]:
## confirm that the sum of the supplies is zero
print("{0}: sum of the supplies is zero".format(sum(supplies)==0))
print("{0}: correct number of supplies".format(len(set(start_nodes)) +1 == len(set(end_nodes)) + 1 == len(supplies)))
print(supplies)

True: sum of the supplies is zero
True: correct number of supplies
[102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -102]


## Set up min cost flow object
In a valid graph:
* The capacity can be greater than the actual flows
* The supply and the sink need to be equal
* All of the supply needs to flow to the sink

In [267]:
min_cost_flow = min_cost_flow.SimpleMinCostFlow()

# Add each arc.
for i in range(len(start_nodes)):
    min_cost_flow.add_arc_with_capacity_and_unit_cost(start_nodes[i], end_nodes[i],
                                                capacities[i], costs[i])
# Add node supplies.
for i in range(len(supplies)):
    min_cost_flow.set_node_supply(i, supplies[i])

In [268]:
min_cost_flow.num_arcs()

1195

## (optional) Output Dotfile of Graph
This dotfile can be loaded into Gephi or output to GraphVis in order to debug and confirm that the solution is acceptable.

In [269]:
def applicant_name(applicant):
    return applicant['First Name'].replace(" ", "_").replace("-", "_") + "_" + applicant['Last Name'].replace(" ", "_").replace("-", "_")

if min_cost_flow.solve() == min_cost_flow.OPTIMAL:
    print('Total cost = ', min_cost_flow.optimal_cost())
    print()

    ## output to dotfile
    with open("data/{0}-allocation-graph.dot".format(int(datetime.datetime.utcnow().timestamp())), "w") as f:

        print("digraph g{", file=f)
        for reviewer in reviewers:
            print("{} [type=reviewer];".format(reviewer['id'].replace("-","_")), file=f)
        for applicant in applicants:
            print("{} [type=applicant];".format(applicant_name(applicant).replace("-","_")), file=f)


        for arc in range(min_cost_flow.num_arcs()):

          # Can ignore arcs leading out of source or into sink.
         if min_cost_flow.tail(arc)!=source and min_cost_flow.head(arc)!=sink:

            # Arcs in the solution have a flow value of 1. Their start and end nodes
            # give an assignment of worker to task.

            if min_cost_flow.flow(arc) > 0:
              applicant = applicants[min_cost_flow.head(arc) - len(reviewers) - 1]
              print('%s -> %s [weight = %d];' % (
                    reviewers[min_cost_flow.tail(arc)-1]['id'].replace("-","_"),
                    (applicant_name(applicant).replace("-","_")),
                    min_cost_flow.unit_cost(arc)), file=f)
        print("}", file=f)
else:
    print('There was an issue with the min cost flow input.')

Total cost =  185



C:\Users\ls13g17\AppData\Local\Temp\ipykernel_18236\4009799678.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  with open("data/{0}-allocation-graph.dot".format(int(datetime.datetime.utcnow().timestamp())), "w") as f:


# Output Applicant Spreadsheet with Assignment Columns
This code takes the applicant dataset and adds two columns to it:
* Reviewer 1
* Reviewer 2

These are the final reviewers. **Reviewers are not listed in any particular order**.

### Add assignment columns to list of dicts

In [270]:
for applicant in applicants:
    if('reviewer 1' in applicant.keys()):
        del applicant['reviewer 1']
    if('reviewer 2'in applicant.keys()):
        del applicant['reviewer 2']

In [271]:
if min_cost_flow.solve() == min_cost_flow.OPTIMAL:
    print('Total cost = ', min_cost_flow.optimal_cost())       
    for arc in range(min_cost_flow.num_arcs()):
     # ignore arcs leading out of source or into sink.
     if min_cost_flow.tail(arc)!=source and min_cost_flow.head(arc)!=sink:
        # Arcs in the solution have a flow value of 1. Their start and end nodes
        # give an assignment of worker to task.
        if min_cost_flow.flow(arc) > 0:
            applicant = applicants[min_cost_flow.head(arc) - len(reviewers) - 1]
            if('reviewer 1' not in applicant.keys()):
                applicant['reviewer 1'] = reviewers[min_cost_flow.tail(arc)-1]['id']
                applicant['priority 1'] = min_cost_flow.unit_cost(arc) 
            else:
                applicant['reviewer 2'] = reviewers[min_cost_flow.tail(arc)-1]['id']
                applicant['priority 2'] = min_cost_flow.unit_cost(arc) 

Total cost =  185


### Check Balance of reviews per faculty

In [272]:
applicant.keys()

dict_keys(['', 'Last Name', 'First Name', '1', '2', '3', '4', 'node_index', 'reviewer 1', 'priority 1', 'reviewer 2', 'priority 2'])

In [273]:
reviewer_reviews = defaultdict(list)
for applicant in applicants:
    reviewer_reviews[applicant['reviewer 1']].append(applicant['Last Name'] + ", " + applicant['First Name'])
    reviewer_reviews[applicant['reviewer 2']].append(applicant['Last Name'] + ", " + applicant['First Name'])

for reviewer, review_names in reviewer_reviews.items():
    print("{0}: {1}".format(reviewer, len(review_names)))

4282ce75-1089-4aac-9bfe-55e65a6312d4: 5
8008686c-7de9-42d8-b27a-2abbebd44e33: 6
42afa2ed-5735-42df-b6a4-e1b6b6e6d151: 2
a13619cd-e1eb-4b49-9a19-e0e423577037: 6
45b7d43f-50d4-49e3-91ca-3a4e2d8bc810: 1
8e826cfa-608b-4409-9203-4ae1b7c5550a: 6
5338d673-a089-4ad4-9d6c-a8184fd16a4d: 6
b44a1a08-334f-4223-93e1-1fde6f090f75: 5
7d50df7d-8b0d-4b43-a90e-375298ecddb3: 6
139b90b3-ca25-4ccb-97c9-0dbf3fa1cc3a: 6
894ed6f3-b99c-44fc-beb5-1e4858e20224: 6
2da9f25d-e3cf-4633-8e15-233157071d75: 6
34a6e61c-0ee3-4803-8728-e6ac8f7582d3: 6
e5203f9d-8874-4f97-882a-fceb05863ea5: 6
bee21ae1-487a-454c-a17d-1d097d9045ab: 6
2fec5177-e9a6-45cc-9dfd-bff6fe5f4bc0: 6
3516c055-4219-4558-bf60-71f2379c8e29: 6
ee2b28ae-0630-489a-9287-d6e677404815: 4
eaba192c-e945-4c8a-a800-47907aa33c50: 6
47d768f6-3d10-4a8a-90ed-31388f9ae08a: 1


# Write to CSV

In [274]:
pd.DataFrame(applicants).to_csv("data/{0}-applicant-reviewer-allocations.csv".format(int(datetime.datetime.utcnow().timestamp())))

C:\Users\ls13g17\AppData\Local\Temp\ipykernel_18236\3579701296.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  pd.DataFrame(applicants).to_csv("data/{0}-applicant-reviewer-allocations.csv".format(int(datetime.datetime.utcnow().timestamp())))
